In [ ]:
# Imports
!pip install cpmpy numpy --quiet

import cpmpy as cp
import numpy as np

(You can ignore google* and tensor* dependency errors on Google Colab.)

## **Session 1: Introduction to CPMpy and Basic Modelling**

In this session, we will be introducing you to constraint programming using the CPMpy framework and working through some modelling exercises. The goal is to help you become familiar with how to formulate/model combinatorial problems using CPMpy.

<!-- <strong>Plan for the session:</strong>
- Introductory Talk (~20 minutes): Introduction to CPMpy, constraint programming, and how combinatorial problems can be represented as models.
- Hands-on Exercises (~2 hours): Practice modelling problems using CPMpy through a series of guided exercises. -->


<strong>If in doubt, quickly check the summary sheet:</strong>
* CPMpy summary sheet: https://cpmpy.readthedocs.io/en/latest/summary.html

<strong>If things are unclear, check the extensive modeling guide:</strong>

* CPMpy modeling documentation: https://cpmpy.readthedocs.io/en/latest/modeling.html

### **Part 1: Short Introduction**

#### **What is Constraint Programming (CP)?**

Constraint Programming is a **declarative paradigm** for solving **combinatorial problems**. In CP, a problem is modelled by defining:

- **Decision Variables**: They represent the solution, as entities that their values need to be determined.
- **Domains**: The set of possible values for the decision variables.
- **Constraints**: Relationships between variables restricting their possible values.
- **Objective (Optional)**: If the problem includes an optimisation objective, we can define an objective to minimize or maximize.

CP allows you to focus on **what** needs to be satisfied, without specifying **how** to solve the problem.

#### **Imperative vs Declarative Programming**

To understand the difference between imperative and declarative programming, let's take a simple example:

We have two variables `x` and `y`, and we want to find their values such that:
- Their sum is equal to 7.
- `x` is greater than `y`.

In [ ]:
# Imperative
x, y = 0, 0

print(f"Imperative Approach: We explicitly assign the variables to specific values: x={x} and y={y}")

print("... and we explicitly reassign it until we find the solution:")
found = False
for x in range(10):
    for y in range(10):
        print(f"x = {x}, y = {y}")
        if x + y == 7 and x > y:
            print(f"\nSolution found: x = {x}, y = {y}")
            found = True
            break
    if found:
        break

In [ ]:
# Declarative

# Define decision variables with domains
x, y = cp.intvar(0, 10, shape=2)

print(f"Declarative Approach: We define the variables x={x} and y={y}.")
print("Notice that we don't explicitly set/assign any values to the variables. We simply define their domains (sets of possible values).")

print("\n... and we model the problem declaratively (i.e. we do not specify how to find the correct assignment, but what the correct assignment should satisfy):\n")

# Model the constraints
model = cp.Model([
    x + y == 7,  # Their sum is 7
    x > y         # x is greater than y
])

print(model)

# Solve the model
print("\n... and we solve the problem:\n")
model.solve()

print(f"Solution: x = {x.value()}, y = {y.value()}")

#### **What is CPMpy?**
CPMpy is a Constraint Programming library in Python, for **modelling** and **solving** combinatorial problems.

**Basic Model Structure**:
1. **Decision Variables**: Defined with `intvar()` for integers or `boolvar()` for booleans.
2. **Constraints**: Applied on variables using comparison operators (`==`, `!=`, `<`, `<=`, etc.) or [global constraints](https://cpmpy.readthedocs.io/en/latest/modeling.html#global-constraints) like `AllDifferent`.
3. **Solving**: Use `solve()` to find a solution.

#### **A Practical Example**

We need to solve a Sudoku puzzle. The goal of Sudoku is to fill a 9x9 grid with numbers so that each row, column and 3x3 section contain all the digits
between 1 and 9.
The given unsolved grid is the following:

```
[  
    [0, 0, 0,  2, 0, 5,  0, 0, 0],
    [0, 9, 0,  0, 0, 0,  7, 3, 0],
    [0, 0, 2,  0, 0, 9,  0, 6, 0],

    [2, 0, 0,  0, 0, 0,  4, 0, 9],
    [0, 0, 0,  0, 7, 0,  0, 0, 0],
    [6, 0, 9,  0, 0, 0,  0, 0, 1],

    [0, 8, 0,  4, 0, 0,  1, 0, 0],
    [0, 6, 3,  0, 0, 0,  0, 8, 0],
    [0, 0, 0,  6, 0, 8,  0, 0, 0]
]
```

The goal is to replace the 0s with acceptable numbers. How to obtain a solution?

In [ ]:
# Data
input_grid = [  # 0 represents empty cells
    [0, 0, 0,  2, 0, 5,  0, 0, 0],
    [0, 9, 0,  0, 0, 0,  7, 3, 0],
    [0, 0, 2,  0, 0, 9,  0, 6, 0],

    [2, 0, 0,  0, 0, 0,  4, 0, 9],
    [0, 0, 0,  0, 7, 0,  0, 0, 0],
    [6, 0, 9,  0, 0, 0,  0, 0, 1],

    [0, 8, 0,  4, 0, 0,  1, 0, 0],
    [0, 6, 3,  0, 0, 0,  0, 8, 0],
    [0, 0, 0,  6, 0, 8,  0, 0, 0]]
given = np.array(input_grid)  # numpy array for easy indexing

# Decision Variables
grid = cp.intvar(1, 9, shape=given.shape)  # Each cell can have a value between 1 and 9

# Model
model = cp.Model()

# Fix the non-zero cells to the given values
model.add(grid[given != 0] == given[given != 0])  # numpy's indexing

# No duplicate number in a row, for all rows
model.add([cp.AllDifferent(row) for row in grid])
# No duplicate number in a column, for all columns
model.add([cp.AllDifferent(col) for col in grid.T])  # numpy's transpose

# No duplicate number in a block, for all blocks
for i in range(0, 9, 3):
    for j in range(0, 9, 3):
        model.add(cp.AllDifferent(grid[i:i+3, j:j+3]))

# For block constraints: Alternative way with list comprehension
model.add([cp.AllDifferent(grid[i:i+3, j:j+3]) for i in range(0, 9, 3) for j in range(0, 9, 3)])

# Solve
model.solve()

# Print solution
print(grid.value())

### **Part 2: Exercises**

#### **1. Five Floors Problem**

Baker, Cooper, Fletcher, Miller, and Smith live on the first five floors of an apartment house.
Baker does not live on the fifth floor.
Cooper does not live on the first floor.
Fletcher does not live on either the fifth or the first floor.
Miller lives on a higher floor than does Cooper.
Smith does not live on a floor adjacent to Fletcher's.
Fletcher does not live on a floor adjacent to Cooper's.
They all live on different floors.
Find the floors where these people live.

In [ ]:
# Decision Variables
"""
TODO: Fill in the lower and upper bounds of the decision variables.

Hints:
- What are the possible values for each variable?
- Which are the possible floors for each person?

https://cpmpy.readthedocs.io/en/latest/modeling.html#decision-variables
"""
lb = None  # lower bound
ub = None  # upper bound

B = cp.intvar(lb, ub)  # Baker
C = cp.intvar(lb, ub)  # Cooper
F = cp.intvar(lb, ub)  # Fletcher
M = cp.intvar(lb, ub)  # Miller
S = cp.intvar(lb, ub)  # Smith

# Since all the decision variables have the same domain, we can also use a more convenient way to define them:
B, C, F, M, S = floors = cp.intvar(lb, ub, shape=5)

# Model
model = cp.Model()

# Constraints
"""
TODO: Fill in the constraints according to the problem description.

Hints:
1. https://cpmpy.readthedocs.io/en/latest/modeling.html#logical-constraints
2. https://cpmpy.readthedocs.io/en/latest/modeling.html#simple-comparison-constraints
3. https://cpmpy.readthedocs.io/en/latest/modeling.html#arithmetic-constraints
4. https://cpmpy.readthedocs.io/en/latest/modeling.html#global-constraints
"""
model.add(None)  # Baker does not live on the fifth floor
model.add(None)  # Cooper does not live on the first floor
model.add(None)  # Fletcher does not live on either the fifth or the first floor
model.add(None)  # Miller lives on a higher floor than does Cooper
model.add(None)  # Smith does not live on a floor adjacent to Fletcher
model.add(None)  # Fletcher does not live on a floor adjacent to Cooper
model.add(None)  # They all live on different floors

# Solve
model.solve()

# Print solution
print(f'Baker lives on floor {B.value()}')
print(f'Cooper lives on floor {C.value()}')
print(f'Fletcher lives on floor {F.value()}')
print(f'Miller lives on floor {M.value()}')
print(f'Smith lives on floor {S.value()}')

#### **2. Bank Card**

My bank card has a 4 digit PIN, abcd. I use the following facts to help me remember it:

- no two digits are the same
- the 2-digit number cd is 3 times the 2-digit number ab
- the 2-digit number da is 2 times the 2-digit number bc

What is my PIN?

In [ ]:
# Decision Variables
"""
TODO: Fill in the lower and upper bound of each digit.
What are the possible values (domain) of each digit?
"""
lb, ub = None, None  # lower bound and upper bound

a, b, c, d = digits = cp.intvar(lb, ub, shape=4)  # The four digits of the PIN

# Model
model = cp.Model()

# Constraints
"""
TODO: Fill in the constraints according to the problem description.
"""
model.add(None)  # no two digits are the same
model.add(None)  # cd is 3 times ab
model.add(None)  # da is 2 times bc

# Solve
model.solve()

# Print solution
print(f"PIN: {a.value()}{b.value()}{c.value()}{d.value()}")

#### **3. Coin Change**

Alice needs to give Bob exactly 1.99 euros in change. She has six different types of coins with the following values: 1, 2, 5, 10, 25, and 50 cents. However, she only has a limited number of each coin type: 20 coins of 1 cent, 10 coins of 2 cents, 15 coins of 5 cents, 8 coins of 10 cents, 4 coins of 25 cents, and 2 coins of 50 cents.

How can Alice give Bob the exact change using the **fewest number of coins possible** while respecting the availability of each coin type?

In [ ]:
# Parameters of the problem
amount = 199  # in cents
coin_types = [1, 2, 5, 10, 25, 50]
available_coins = [20, 10, 15, 8, 4, 2]

# Decision Variables - TODO


# Model
model = cp.Model()

# Constraints - TODO


# Objective: Minimize the number of coins - TODO
model.minimize(None)

# Solve
model.solve()

# Print solution - TODO
print(  )

#### **4. Magic Square**

A magic square is an $n \times n$ grid such that each cell contains a different integer from 1 to $n^2$ and the sum of the integers in each row, column and diagonal is equal.
Find a magic square for any size $n$, knowing that the sum of each row, column and diagonal has to be equal to $n(n^2 + 1)/2$ (an integer: div
operator). Remember there is no magic square for size 2.

<details>
  <summary>Click to reveal hint</summary>
  
```
"""
An example to show how to use indexing for diagonals in matrices.

Suppose that we have a grid of integers, and we require every number in the main
diagonal to be different, and also every number in the secondary diagonal to be
different.
"""

# Parameters
n = 50

# Decision Variables
grid = cp.intvar(0, n, shape=(n, n))

# Constraints
model = cp.Model()

model += cp.AllDifferent(grid[i, i] for i in range(n))  # Every number in the main diagonal is different
model += cp.AllDifferent(grid[i, n - 1 - i] for i in range(50))  # Every number in the secondary diagonal is different

# Solve
model.solve()
```
  
</details>

In [ ]:
def magic_square(n):

  assert n != 2, "Magic square is not defined for size 2"

  """
  TODO: define the magic sum as a function of n.
  Use // operator to get the integer part of a division (div operator).
  """
  magic_sum = None  # sum of each row, column and diagonal

  # Decision Variables - TODO


  # Model
  model = cp.Model()

  # Constraints

  """
  TODO: Fill in the constraints according to the problem description.

  Hints:
  1. https://cpmpy.readthedocs.io/en/latest/api/expressions/globalconstraints.html#cpmpy.expressions.globalconstraints.AllDifferent
  2. https://cpmpy.readthedocs.io/en/latest/api/expressions/python_builtins.html#cpmpy.expressions.python_builtins.sum
  """

  # All numbers in the magic square must be different


  # The sum of the numbers in each row must be equal to the magic sum


  # The sum of the numbers in each column must be equal to the magic sum


  # The sum of the numbers in the main diagonal must be equal to the magic sum


  # The sum of the numbers in the other diagonal must be equal to the magic sum


  # Solve
  model.solve()

  # Print solution - TODO
  print(  )


# Test
magic_square(5)

#### **5. Movie Scheduling**:

Consider the following scheduling problem. Imagine you are a highly-in-
demand actor, who has been presented with offers to star in n different movie
projects under development. Each offer comes specified with the first and last day of filming. To take the job, you must commit to being available throughout this entire period. Thus, you cannot simultaneously accept two jobs whose intervals overlap.
For an artist such as yourself, the criteria for job acceptance is clear: you want to make as much money as possible. Because each of these films pays the same fee per film, this implies you seek the largest possible set of jobs (intervals) such that no two of them conflict with each other.

Here is the list of movies along with their first and last day of filming:
```
movies = [  # title, start, end
    ["Tarjan of the Jungle", 4, 13],
    ["The Four Volume Problem", 17, 27],
    ["The President's Algorist", 1, 10],
    ["Steiner's Tree", 12, 18],
    ["Process Terminated", 23, 30],
    ["Halting State", 9, 16],
    ["Programming Challenges", 19, 25],
    ["Discrete Mathematics", 2, 7],
    ["Calculated Bets", 26, 31]
]
```

Which movies would you choose to maximize your income?

In [ ]:
# Decision Variables - TODO


# Model
model = cp.Model()

# Constraints - TODO


# Objective - TODO
model.maximize(None)

# Solve
model.solve()

# Print solution - TODO
print(   )

#### **6. Robbery**

Following a robbery, an inspector interviewed six suspects. The getaway car had been barely big enough to hold two, so she reckoned that at least four of them were innocent. She also supposed that the innocent ones would tell the truth, while the guilty one or ones would lie. What they actually said was:
- ARTIE: "It wasn't me."
- BILL: "Crackitt was in it up to his neck."
- CRACKITT: "No I wasn't."
- DODGY: "If Crackitt did it, Bill did it with him."
- EDGY: "Nobody did it alone."
- FINGERS: "It was Artie and Dodgy together."

Who is guilty?

<details>
  <summary>Click to reveal hint</summary>

  **Logic Truth Table:**
  
  | p     | q     | ¬p   | p ∧ q | p ∨ q | p → q | p ↔ q |
  |-------|-------|------|-------|-------|-------|-------|
  | True  | True  | False| True  | True  | True  | True  |
  | True  | False | False| False | True  | False | False |
  | False | True  | True | False | True  | True  | False |
  | False | False | True | False | False | True  | True  |

  **Explanation of Symbols:**
  - ¬p: Negation (NOT p)
  - p ∧ q: Conjunction (AND)
  - p ∨ q: Disjunction (OR)
  - p → q: Implication (If p then q)
  - p ↔ q: Biconditional (p if and only if q)

</details>

In [ ]:
# Decision Variables - TODO


# Model
model = cp.Model()

# Constraints - TODO


# Solve
model.solve()

# Print solution - TODO
print(  )

#### **Extra Exercises**

> The following exercises are **extra practice** problems. They will **not be covered** during the regular exercise sessions. However, they offer a great opportunity to practice more. You are encouraged to try them at home to challenge yourself further!


##### **Extra 1: Send More Money**

The "Send More Money" puzzle is a classic cryptarithmetic problem where each letter represents a unique digit. The goal is to assign digits to the letters such that the following equation holds true:
```
   SEND
 + MORE
 ------
  MONEY
```
Each letter must be assigned a unique digit from 0 to 9, and the first letter of each word cannot be zero. What is the digit assigned to each letter?

In [ ]:
# Decision Variables - TODO


# Model
model = cp.Model()

# Constraints - TODO


# Solve
model.solve()

# Print solution - TODO
print(  )

##### **Extra 2: Minesweeper**

Numbers and mines are randomly distributed in a given board.
The number on a cell shows the exact number of mines around this cell in all 8 directions: top, top-right, right, bottom-right, bottom, bottom-left, left, top-left.

The task is to determine which cells contain mines, the initial board is given below.

<details>
  <summary>Click to reveal hint</summary>

```
"""
An example to show how to find neighbors (top, right, bottom, left) of a cell in a grid.
"""

n = 4  # 4x4 grid
grid = cp.boolvar(shape=(n, n))

model = cp.Model()

for i in range(n):
    for j in range(n):
        # Collect neighbors for each cell
        neighbors = []
        for a, b in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            if 0 <= i + a < n and 0 <= j + b < n:
                neighbors.append(grid[i + a, j + b])

        print(f"Neighbors of cell {grid[i, j]}: {neighbors}")

print("The grid:")
print(grid)
```

</details>

In [ ]:
# Data
X = -1
game = np.array([  # 0-8: number of mines around, -1: not opened
    [2, 3, X, 2, 2, X, 2, 1],
    [X, X, 4, X, X, 4, X, 2],
    [X, X, X, X, X, X, 4, X],
    [X, 5, X, 6, X, X, X, 2],
    [2, X, X, X, 5, 5, X, 2],
    [1, 3, 4, X, X, X, 4, X],
    [0, 1, X, 4, X, X, X, 3],
    [0, 1, 2, X, 2, 3, X, 2]
])
rows, cols = game.shape


# Decision Variables - TODO


# Model
model = cp.Model()

# Constraints - TODO


# Solve
model.solve()

# Print solution - TODO
print(  )